In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import pandas as pd
import os


In [5]:
df = pd.read_table("../data/data.txt")
df.head()

,0
0,ሐምሌ 7 ቀን 1971 ዓ.ም. ዕለተ ቅዳሜ ከታላቁ ቤተ መንግሥት ምድር ቤ...
1,ከቀኑ 6፡00 ሰዓት ላይ ከእስረኛ ክፍል ቁጥር 2 የሚፈለጉት ሰዎች ሥም ...
2,”ልጅ ካሳ ወ/ማርያም፤ ብ/ጄ ሳሙኤል አበበ!” ከዚያም ”ወደ ውጭ ትፈለጋ...
3,በዚህ ጊዜ ልጅ ካሳ ወ/ማርያም ወደ ፓትርያርክ ቴዎፍሎስ ቀርበው፤
4,”አባታችን ሊገሉን ነው፤ ይፍቱኝ!” አሏቸው። እሳቸውም ”እግዚአብሔር ይፍ...


In [10]:
df.shape

(1146, 1)

In [11]:
#converting to list
lines = []
for index, row in df.iterrows():
    lines.append(row['0'])

lines[:5]

['ሐምሌ 7 ቀን 1971 ዓ.ም. ዕለተ ቅዳሜ ከታላቁ ቤተ መንግሥት ምድር ቤት እስረኞች መካከል የክርስትና እምነት ተከታዮች የሆኑት በጠዋት ተነስተው በአባቶች መሪነት ጸሎት አድርሰዋል። የቅድስት ሥላሴ ካቴድራል ደውል ድምፅ ሲያስተጋባ ሰምተዋል። ፈጣሪያቸውን አውቀው በድፍረት ሳያውቁ በስህተት የፈፀሙትን በደል ይቅር እንዲላቸው ተማጽነዋል። ከጸሎት በኋላ እስረኞቹ ጧት ፀሐይ ለመሞቅና አየር ለመቀበል የተፈቀደላቸው ለ45 ደቂቃ ጊዜ አጠናቀው ወደየክፍሎቻው ተመልሰው ተቆልፎባቸዋል። የሐምሌ ፀሐይ በቀዝቃዛው በታላቁ ቤተ መንግሥት ምድር ቤት ውስጥ የታጎሩትን ታላላቅ ኢትዮጵያውያን ለማጽናናት የተላከች ይመስላል፤ ሰማዩን በጋረደው ደመና መካከል ብልጭ ድርግም ትላለች።',
 'ከቀኑ 6፡00 ሰዓት ላይ ከእስረኛ ክፍል ቁጥር 2 የሚፈለጉት ሰዎች ሥም ዝርዝር በመጥራት፤ ”ራቅ ወዳለ ቦታ ስለምትሄዱ ብርድ ልብስ እና ሶፍት ያዙ” ተብሎ ተነገራቸው … በመቀጠልም ወደ ክፍል 1 በእስረኞች አለቃ አማካኝነት የሚፈልጉትን የእስረኛ ሥም መጥራት ጀመሩ …',
 '”ልጅ ካሳ ወ/ማርያም፤ ብ/ጄ ሳሙኤል አበበ!” ከዚያም ”ወደ ውጭ ትፈለጋላችሁ፤ የምትሄዱበት ራቅ ወደአለ ቦታ ስለሆነ የብርድ ልብስና የሽንት ቤት ወረቀት ያዙ ተብሏል” አሉ።',
 'በዚህ ጊዜ ልጅ ካሳ ወ/ማርያም ወደ ፓትርያርክ ቴዎፍሎስ ቀርበው፤',
 '”አባታችን ሊገሉን ነው፤ ይፍቱኝ!” አሏቸው። እሳቸውም ”እግዚአብሔር ይፍታ፤ አይዞህ አትፍራ ሥጋህን እንጂ ነፍስህን አይገድሏትም” አሏቸው። ልጅ ካሳ ወ/ማርያም መስቀል ተሳልመው ከወጡ ከጥቂት ጊዜ በኋላ ፓትርያርክ ቴዎፍሎስ ስማቸው ተጠራ። ፓትያርኩ ስማቸው ሲጠራ ከደረጃው ሥር ወዳለው መኝታቸው በመሔድ የሚፈልጉትን ልብስ መርጠው ከለበሱ በኋላ ጸሎት ማድረስ ጀመሩ፤ በዚህ ሁኔታ

In [13]:
#converting list to string
data = ""
for i in lines:
    data = ' '.join(lines)

data[:50]

'ሐምሌ 7 ቀን 1971 ዓ.ም. ዕለተ ቅዳሜ ከታላቁ ቤተ መንግሥት ምድር ቤት እስ'

In [14]:
#replacing unnecessary characters with space
data = data.replace('\n', ' ').replace('.', ' ').replace(
    ',', ' ').replace('"', ' ').replace(';', ' ').replace('!', ' ')
data = data.replace('?', ' ').replace('(', ' ').replace(')', ' ').replace(
    '--', ' ').replace('።', ' ').replace('፣', ' ')
data = data.replace(':', ' ').replace(
    '፤', ' ').replace('…', ' ').replace('”', ' ')

#remove unnecessary spaces
data = data.split()
data = ' '.join(data)
data[:50]


'ሐምሌ 7 ቀን 1971 ዓ ም ዕለተ ቅዳሜ ከታላቁ ቤተ መንግሥት ምድር ቤት እስረ'

In [16]:
print(len(data.split()))

22001


In [18]:
len(data)

111487

In [21]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

#saving the tokenizer for prediction function
pickle.dump(tokenizer, open('../token/token.pkl', 'wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]


[520, 270, 29, 2783, 57, 51, 1448, 681, 2784, 227, 67, 333, 22, 271, 59]